In [29]:

import sys, os

import importlib, time
import traceback
import numpy as np
import time

import torch
import torch.optim as optim
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.data import DataLoader
from humor.body_model.body_model import BodyModel

from humor.utils.config_new import ConfigParser
from humor.utils.logging import Logger, class_name_to_file_name, mkdir, cp_files
from humor.utils.torch import get_device, save_state, load_state
from humor.utils.stats import StatTracker
from humor.models.model_utils import step

import smplx
import torch
from smplx import SMPL, SMPLH, SMPLX

import pickle

NUM_WORKERS = 4
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
config_path = r"configs\train_diffusion_transformer.yaml"
config_parser_yaml = ConfigParser(config_path)
args_obj, _ = config_parser_yaml.parse('train')
# See config
dict_attr = ['base_dict', 'model_dict', 'dataset_dict', 'loss_dict']
for attr in dict_attr:
    print(f"{attr}: {getattr(args_obj, attr)}")
args = args_obj.base

Using default: {'ckpt', 'load_optim', 'decay', 'beta2', 'eps', 'use_adam', 'beta1'}
Using default: {'detach_sched_samp', 'output_delta', 'model_use_smpl_joint_inputs'}
Using default: {'frames_out_step_size', 'data_noise_std', 'splits_path'}
Using default: set()
base_dict: {'dataset': 'AmassDiscreteDataset', 'model': 'HumorDiffusionTransformer', 'loss': 'DiffusionLoss', 'out': './out/humordiffusiontransformer_train', 'ckpt': None, 'gpu': 0, 'batch_size': 64, 'print_every': 10, 'epochs': 200, 'val_every': 2, 'save_every': 25, 'lr': 0.0001, 'beta1': 0.9, 'beta2': 0.999, 'eps': 1e-08, 'sched_milestones': [20, 40, 80], 'sched_decay': 0.5, 'decay': 0.0, 'load_optim': True, 'use_adam': False, 'sched_samp_start': -1, 'sched_samp_end': -2}
model_dict: {'out_rot_rep': 'aa', 'in_rot_rep': 'mat', 'latent_size': 128, 'steps_in': 1, 'output_delta': True, 'model_data_config': 'smpl+joints+contacts', 'detach_sched_samp': True, 'model_use_smpl_joint_inputs': False, 'pose_token_dim': 64, 'diffusion_base

In [3]:
from humor.models.humor_diffusion_transformer import HumorDiffusionTransformer
print(f"Model: {args_obj.model_dict}")
model = HumorDiffusionTransformer(**args_obj.model_dict,
                                  vae_ckpt=r'out\motion_vae\20250506_014121\checkpoints\best_model.pth',
                                  vae_cfg=r'configs\train_motion_vae.yaml',
                                  model_smpl_batch_size=args.batch_size,)

Model: {'out_rot_rep': 'aa', 'in_rot_rep': 'mat', 'latent_size': 128, 'steps_in': 1, 'output_delta': True, 'model_data_config': 'smpl+joints+contacts', 'detach_sched_samp': True, 'model_use_smpl_joint_inputs': False, 'pose_token_dim': 64, 'diffusion_base_dim': 256, 'nhead': 4, 'num_layers': 6, 'dim_feedforward': 1024, 'dropout': 0.1, 'cfg_scale': 4.0, 'cond_drop_prob': 0.1, 'use_mean_sample': True}
Using default: {'ckpt', 'load_optim', 'decay', 'beta2', 'eps', 'use_adam', 'beta1'}
Using default: {'output_delta', 'model_use_smpl_joint_inputs', 'detach_sched_samp'}
Using default: {'frames_out_step_size', 'data_noise_std', 'splits_path'}
Using default: {'kl_loss_cycle_len', 'smpl_vert_consistency_loss'}


e:\workspace\Motion\humor\humor\utils\torch.py:47: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  full_checkpoint_dict = torch.load(load_path, map_location=map_location)


In [4]:
device = get_device(args.gpu)

Using detected GPU...


In [5]:
from humor.datasets. amass_discrete_dataset import AmassDiscreteDataset

train_dataset = AmassDiscreteDataset(split='train', **args_obj.dataset_dict,)
# create loaders

train_loader = DataLoader(train_dataset,
                            batch_size=4,
                            shuffle=False, 
                            num_workers=NUM_WORKERS,
                            pin_memory=True)

Loading data from../datasets/AMASS/amass_processed
Logger must be initialized before logging!
This split contains 2425 sequences (that meet the duration criteria).
Logger must be initialized before logging!
The dataset contains 106399 sub-sequences in total.
Logger must be initialized before logging!


In [8]:
for i, data in enumerate(train_loader):
    data_in, data_out, meta = data
    for k, v in data_in.items():
        print(k, v.shape)
    break

pose_body torch.Size([4, 10, 1, 189])
root_orient torch.Size([4, 10, 1, 9])
root_orient_vel torch.Size([4, 10, 1, 3])
trans torch.Size([4, 10, 1, 3])
trans_vel torch.Size([4, 10, 1, 3])
joints torch.Size([4, 10, 1, 22, 3])
joints_vel torch.Size([4, 10, 1, 22, 3])
contacts torch.Size([4, 10, 1, 9])


In [20]:
path=r'body_models\smplh\neutral\model.npz'
neutral_bm = BodyModel(bm_path=path, num_betas=16, batch_size=args.batch_size)

In [21]:
for i, data in enumerate(train_loader):
    data_in, data_out, meta = data
    pose = data_in['pose_body']
    beta = meta['betas']
    root_orient = data_in['root_orient']
    trans = data_in['trans']
    break

In [41]:
path= r"E:\workspace\Motion\AMASS\amass_processed\DanceDB\20140805_NikosMichael\Nikos_Zeibekiko_v2_C3D_poses_5170_frames_30_fps.npz"

data = np.load(path, allow_pickle=True)

pose_body = data['pose_body'] # N x 63
root_orient = data['root_orient'] # N x 3
pose_hand = np.zeros((pose_body.shape[0], 6))

pose = np.concatenate([root_orient, pose_body, pose_hand], axis=1) # N x 72
trans = data['trans']

In [42]:
pkl_dict = {
    "smpl_poses": pose,
    "smpl_trans": trans,
    "smpl_scaling": np.ones((pose.shape[0], 1)),
    "smpl_betas": np.zeros((pose.shape[0], 16)),
}
name = path.split('\\')[-1].split('.')[0]

save_path = f"out/vis/{name}.pkl"

with open(save_path, 'wb') as f:
    pickle.dump(pkl_dict, f)


In [44]:
pose_mask = np.zeros((pose.shape[0], 72))
for i in range(pose.shape[1]):
    if i in [42, 43, 44, 51, 52, 53, 57, 58, 59, 63, 64, 65]:
        continue
    pose_mask[:, i] = pose[:, i]
    
pkl_mask_dict = {
    "smpl_poses": pose_mask,
    "smpl_trans": trans,
    "smpl_scaling": np.ones((pose.shape[0], 1)),
    "smpl_betas": np.zeros((pose.shape[0], 16)),
}

save_path = f"out/vis/{name}_mask.pkl"
with open(save_path, 'wb') as f:
    pickle.dump(pkl_mask_dict, f)

In [ ]:
noise = np.random.randn(pose.shape[0], 72) * 0.8
noise_dict = {
    "smpl_poses": noise + pose,
    "smpl_trans": trans,
    "smpl_scaling": np.ones((pose.shape[0], 1)),
    "smpl_betas": np.zeros((pose.shape[0], 16)),
}
save_path = f"out/vis/{name}_noise.pkl"
with open(save_path, 'wb') as f:
    pickle.dump(noise_dict, f)